In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pipeline
import numpy as np
import pandas as pd

In [3]:
kdd = pipeline.KDD1999()

In [4]:
kdd.report_training_data()

shape
(98525, 42)
Labels
          counts       pct
normal.    97278  0.987343
ipsweep.    1247  0.012657


In [5]:
kdd.baseline()

Confusion matrix
                  counts       pct
predicted actual                  
False     False      798  0.008099
          True       449  0.004557
True      False      449  0.004557
          True     96829  0.982786
Scores
           baseline    random  relative
f1         0.995384  0.987335  0.008152
ap         0.995347  0.987330  0.008121
precision  0.995384  0.987338  0.008149
recall     0.995384  0.987387  0.008100


### modAL

In [6]:
# !pip install modAL

In [7]:
n_sample = 1000
indices = np.random.choice(range(len(kdd.x)),
                           size=n_sample,
                           replace=False)
x_sample = kdd.x.iloc[indices, :]
y_sample = kdd.y.iloc[indices]

In [8]:
kdd.y.value_counts(normalize=True)

True     0.987343
False    0.012657
Name: label, dtype: float64

In [9]:
y_sample.value_counts(normalize=True)

True     0.982
False    0.018
Name: label, dtype: float64

In [15]:
from modAL import uncertainty
from sklearn.ensemble import RandomForestClassifier

In [16]:
clf = RandomForestClassifier()

In [17]:
from numpy import ndarray

isinstance(kdd.y.values, ndarray)

True

In [18]:
uncertainty.entropy_sampling(clf, kdd.x.values)

(array([0]),
 array([[8.00e+00, 0.00e+00, 5.45e+03, 9.00e+00, 0.00e+00, 0.00e+00,
         1.10e-01, 1.00e+00, 0.00e+00, 9.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00,
         1.81e+02, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00,
         1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
         0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00]]))

In [19]:
isinstance(RandomForestClassifier(), BaseEstimator)

True

In [21]:
from modAL.models import ActiveLearner
from sklearn.ensemble import RandomForestClassifier
from modAL.uncertainty import entropy_sampling

# initializing the learner
learner = ActiveLearner(
    estimator=RandomForestClassifier(n_estimators=100),
    # query_strategy=entropy_sampling,
    X_training=x_sample.values, y_training=y_sample.values,
)

In [22]:
learner.score(kdd.x, kdd.y)

0.9989748794722152

In [23]:
idx, inst = learner.query(kdd.x.values)

In [24]:
idx

array([66836])

In [25]:
kdd.y.iloc[idx].astype(np.float).values  #.resahpe(1, -1)

array([1.])

In [26]:
kdd.x.iloc[idx, :].values.shape

(1, 82)

In [30]:
kdd.y.iloc[0]

True

In [27]:
learner.teach(kdd.x.iloc[idx, :].values, kdd.y.iloc[idx].values)

In [28]:
kdd.x.values[0]

array([8.00e+00, 0.00e+00, 5.45e+03, 9.00e+00, 0.00e+00, 0.00e+00,
       1.10e-01, 1.00e+00, 0.00e+00, 9.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00,
       1.81e+02, 8.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.00e+00, 0.00e+00,
       1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 1.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
       0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00])